In [6]:
import glob
import re
import ast
import os
import networkx as nx
from collections import defaultdict

def read_adjacency_from_file(path):
    with open(path, 'r') as f:
        content = f.read()
    match = re.search(r"Room Adjacency List:\s*(\{.*\})", content)
    if not match:
        raise ValueError(f"Could not find 'Room Adjacency List' in {path}.")
    return ast.literal_eval(match.group(1))

def build_graph(adjacency_dict):
    G = nx.Graph()
    for room, neighbors in adjacency_dict.items():
        for nbr, w in neighbors.items():
            G.add_edge(room, nbr, weight=w)
    return G

def largest_connected_subgraph(G):
    if nx.is_connected(G):
        return G
    largest_cc = max(nx.connected_components(G), key=len)
    return G.subgraph(largest_cc).copy()

def compute_indices(G):
    V = G.number_of_nodes()
    E = G.number_of_edges()
    EI = V + E
    MI = (E - V + 1) / (2.0 * V - 5) if (2 * V - 5) > 0 else 0.0
    H = largest_connected_subgraph(G)
    ASPLI = nx.average_shortest_path_length(H, weight='weight')
    return EI, MI, ASPLI

def compute_difficulty(ei, mi, aspli,
                       ei_min, ei_max,
                       mi_min, mi_max,
                       aspli_min, aspli_max):
    def safe_norm(x, xmin, xmax):
        return 0.0 if xmax == xmin else (x - xmin) / (xmax - xmin)
    EI_n    = safe_norm(ei,    ei_min,    ei_max)
    MI_n    = safe_norm(mi,    mi_min,    mi_max)
    ASPLI_n = safe_norm(aspli, aspli_min, aspli_max)
    D = (EI_n + MI_n + ASPLI_n) / 3.0
    if D < 0.33:
        label = "easy"
    elif D < 0.66:
        label = "medium"
    else:
        label = "hard"
    return label, D


In [7]:
def run(pattern, error_log_path, summary_file):
    info_files = glob.glob(pattern)

    file_indices = {}
    open(error_log_path, 'w').close()

    # First pass: compute graph metrics
    for info_path in info_files:
        try:
            adj = read_adjacency_from_file(info_path)
            G   = build_graph(adj)
            file_indices[info_path] = compute_indices(G)
        except Exception as e:
            with open(error_log_path, 'a') as ef:
                ef.write(f"{os.path.dirname(info_path)} -> {e}\n")
            continue

    if not file_indices:
        print("No valid graphs found. Exiting.")
        return

    # Global min/max for normalization
    all_EI    = [v[0] for v in file_indices.values()]
    all_MI    = [v[1] for v in file_indices.values()]
    all_ASPLI = [v[2] for v in file_indices.values()]
    EI_min, EI_max       = min(all_EI),    max(all_EI)
    MI_min, MI_max       = min(all_MI),    max(all_MI)
    ASPLI_min, ASPLI_max = min(all_ASPLI), max(all_ASPLI)

    difficulty_map   = defaultdict(list)   # difficulty -> list of (path, score)

    # Second pass: compute difficulty and overwrite score in info.txt
    for path, (ei, mi, aspli) in file_indices.items():
        diff, score = compute_difficulty(
            ei, mi, aspli,
            EI_min, EI_max,
            MI_min, MI_max,
            ASPLI_min, ASPLI_max
        )
        difficulty_map[diff].append((path, score))

        # Clean up old entries if exist
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        content = re.sub(r"(?m)^score:.*\n?", "", content)
        content = re.sub(r"(?m)^Graph difficulty:.*\n?", "", content)
        content = re.sub(r"(?m)^Raw Indices:.*\n?", "", content)

        # Append new info
        with open(path, 'w', encoding='utf-8') as f:
            f.write(content.strip() + "\n")
            f.write(f"Raw Indices: EI={ei}, MI={mi:.4f}, ASPLI={aspli:.4f}\n")
            f.write(f"score:  {score:.6f}\n")
            f.write(f"Graph difficulty: {diff}\n\n")

    # Write summary file
    with open(summary_file, 'w') as sf:
        sf.write("Summary of all floorplans by difficulty:\n\n")
        for label in ["easy", "medium", "hard"]:
            entries = sorted(difficulty_map.get(label, []), key=lambda x: x[1])
            sf.write(f"{label}: {len(entries)} file(s)\n")
            for path, score in entries:
                sf.write(f"   {path}  (score: {score:.6f})\n")
            sf.write("\n")

    print(f"\n✅ Wrote summary to: {summary_file}")
    print(f"⚠️ Errors (if any) are in: {error_log_path}")

# Run

In [26]:
pattern = '../cubicasa5k-3493-cubigraph/*/*/info.txt'
error_log_path = "./error-3493.txt"
summary_file = "./graph_complexity_info_3493.txt"
run(pattern, error_log_path, summary_file)


✅ Wrote summary to: ./graph_complexity_info_3493.txt
⚠️ Errors (if any) are in: ./error-3493.txt


In [27]:
import os
import shutil

# 변경할 두 루트 디렉터리 경로
root_650 = '../cubicasa5k-650'
root_3493 = '../cubicasa5k-3493-cubigraph'

# 650쪽 카테고리별 폴더 순회
num = 1
for category in os.listdir(root_650):
    dir650_cat = os.path.join(root_650, category)
    dir3493_cat = os.path.join(root_3493, category)
    if not os.path.isdir(dir650_cat) or not os.path.isdir(dir3493_cat):
        continue

    # 각 카테고리 폴더 내 숫자별 하위 폴더 순회
    for sub in os.listdir(dir650_cat):
        path650 = os.path.join(dir650_cat, sub, 'info.txt')
        path3493 = os.path.join(dir3493_cat, sub, 'info.txt')

        # 두 파일 모두 존재하면 덮어쓰기
        if os.path.isfile(path650) and os.path.isfile(path3493):
            shutil.copyfile(path3493, path650)
            print(f'{num} Overwrote: {path3493} ← {path650}')
            num +=1 


1 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/14267/info.txt ← ../cubicasa5k-650/high_quality/14267/info.txt
2 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/12190/info.txt ← ../cubicasa5k-650/high_quality/12190/info.txt
3 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/11186/info.txt ← ../cubicasa5k-650/high_quality/11186/info.txt
4 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/9000/info.txt ← ../cubicasa5k-650/high_quality/9000/info.txt
5 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/6211/info.txt ← ../cubicasa5k-650/high_quality/6211/info.txt
6 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/9857/info.txt ← ../cubicasa5k-650/high_quality/9857/info.txt
7 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/14692/info.txt ← ../cubicasa5k-650/high_quality/14692/info.txt
8 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/1468/info.txt ← ../cubicasa5k-650/high_quality/1468/info.txt
9 Overwrote: ../cubicasa5k-3493-cubigraph/high_quality/991/info.

In [46]:
import os
from collections import defaultdict

def parse_info_file(info_path):
    difficulty = None
    score = None
    with open(info_path, 'r') as f:
        for line in f:
            line = line.strip().lower()
            if line.startswith("graph difficulty:"):
                difficulty = line.split(":", 1)[1].strip()
            elif line.startswith("score:"):
                try:
                    score = float(line.split(":", 1)[1].strip())
                except ValueError:
                    pass
    return difficulty, score

def generate_summary(root_dir, output_file):
    summary = defaultdict(list)
    categories = ["colorful", "high_quality", "high_quality_architectural"]

    for category in categories:
        cat_path = os.path.join(root_dir, category)
        #print("cat_path: ", cat_path)
        if not os.path.isdir(cat_path):
            print(f"Warning: {cat_path} not found, skipping.")
            continue

        for sub in os.listdir(cat_path):
            sub_path = os.path.join(cat_path, sub)
            #print("sub_path: ", sub_path)
            if not os.path.isdir(sub_path):
                continue  # Skip files like .DS_Store
            
            info_path = os.path.join(sub_path, "info.txt")
            if not os.path.isfile(info_path):
                print(f"Warning: {info_path} not found, skipping.")
                continue
            print("info_path: ", info_path)
            difficulty, score = parse_info_file(info_path)
            print(difficulty, score)
            if difficulty and score is not None:
                rel_path = f"../cubicasa5k-3493-cubigraph/{category}/{sub}/info.txt"
                summary[difficulty].append((rel_path, score))

    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    print(summary)
    with open(output_file, 'w') as out:
        out.write("Summary of all floorplans by difficulty:\n\n")
        for diff in sorted(summary):
            entries = summary[diff]
            out.write(f"{diff}: {len(entries)} file(s)\n")
            for path, score in entries:
                out.write(f"   {path}  (score: {score:.6f})\n")
            out.write("\n")

# Final path to run with
root_650 = "/Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650"
output_file = "./graph_complexity_info_650.txt"

generate_summary(root_650, output_file)


info_path:  /Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650/colorful/13278/info.txt
easy 0.235006
info_path:  /Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650/colorful/13019/info.txt
easy 0.266641
info_path:  /Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650/colorful/476/info.txt
easy 0.174483
info_path:  /Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650/colorful/12214/info.txt
easy 0.287666
info_path:  /Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650/colorful/1570/info.txt
easy 0.245995
info_path:  /Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650/colorful/13138/info.txt
medium 0.358788
info_path:  /Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650/colorful/268/info.txt
medium 0.372305
info_path:  /Users/jeongahlee/Documents/github folder/CubiGraph5K/cubicasa5k-650/colorful/3651/info.txt
easy 0.245518
info_path:  /Users/jeongahlee/Documents/github fol